In [1]:
from utils import roi_utils
import numpy as np
import copy 
import os

from utils import nsd_utils, default_paths

import pandas as pd
    

In [2]:
for ss in np.arange(1,9):
    roi_def = roi_utils.nsd_roi_def(subject=ss)
    n_each = roi_def.get_sizes()
    if ss==1:
        nvox = np.zeros((8, roi_def.n_rois),dtype=int)
        
    nvox[ss-1,:] = n_each
nvox = np.concatenate([nvox, np.sum(nvox, axis=0, keepdims=True)], axis=0)
roi_df = pd.DataFrame(data=nvox, columns=roi_def.roi_names, \
                      index=['%d'%ss for ss in np.arange(1,9)] + ['total'])

In [3]:
roi_df.iloc[:,0:12]

,V1,V2,V3,hV4,VO1-2,PHC1-2,LO1-2,TO1-2,V3ab,IPS0-1,IPS2-5,SPL1
1,2392,2096,1674,721,482,382,488,339,965,1068,1087,164
2,1630,1509,1574,522,424,380,506,419,844,1267,912,201
3,2275,1732,1457,524,498,351,530,522,829,1114,1190,220
4,1526,1301,1172,610,462,351,419,369,710,1230,1043,149
5,1705,1455,1350,704,591,420,345,311,706,919,806,175
6,1771,1739,1938,845,695,485,715,403,883,1135,939,158
7,1926,1475,1348,478,459,305,406,391,739,855,598,142
8,1791,1494,1445,543,489,395,442,504,654,1075,1016,187
total,15016,12801,11958,4947,4100,3069,3851,3258,6330,8663,7591,1396


In [4]:
roi_df.iloc[:,12:24]

,FEF,OPA,PPA,RSC,OFA,FFA-1,FFA-2,mTL-faces,aTL-faces,EBA,FBA-1,FBA-2
1,72,1611,1033,566,355,484,310,0,159,2971,396,430
2,97,1381,994,813,441,340,529,0,275,3439,0,1217
3,74,1330,1269,838,701,697,396,0,159,3518,875,457
4,107,1235,960,813,602,479,463,163,0,3288,137,524
5,64,1332,1221,771,782,452,455,103,148,4587,460,508
6,93,1362,1229,845,519,341,485,0,80,4126,985,301
7,65,1083,912,694,316,346,138,80,231,3062,0,552
8,93,1360,961,799,331,556,648,273,203,3184,433,488
total,665,10694,8579,6139,4047,3695,3424,619,1255,28175,3286,4477


In [5]:
# which ones are missing? will skip these if missing for any subjects.

rois_empty = np.any(np.array(roi_df)==0, axis=0)
print(np.where(rois_empty))
print(np.array(roi_def.roi_names)[rois_empty])

(array([19, 20, 22, 24]),)
['mTL-faces' 'aTL-faces' 'FBA-1' 'mTL-bodies']


In [23]:
for ss in np.arange(1,9):
    roi_def = roi_utils.nsd_roi_def(subject=ss, skip_areas = [4,5,7,11,12, 19,20, 22, 24])
    n_each = roi_def.get_sizes()
    if ss==1:
        nvox = np.zeros((8, roi_def.n_rois),dtype=int)
        
    nvox[ss-1,:] = n_each
nvox = np.concatenate([nvox, np.sum(nvox, axis=0, keepdims=True)], axis=0)
roi_df = pd.DataFrame(data=nvox, columns=roi_def.roi_names, \
                      index=['%d'%ss for ss in np.arange(1,9)] + ['total'])

In [27]:
roi_df

,V1,V2,V3,hV4,LO1-2,V3ab,IPS0-1,IPS2-5,OPA,PPA,RSC,OFA,FFA-1,FFA-2,EBA,FBA-2
1,2392,2096,1674,721,488,965,1068,1087,1611,1033,566,355,484,310,2971,430
2,1630,1509,1574,522,506,844,1267,912,1381,994,813,441,340,529,3439,1217
3,2275,1732,1457,524,530,829,1114,1190,1330,1269,838,701,697,396,3518,457
4,1526,1301,1172,610,419,710,1230,1043,1235,960,813,602,479,463,3288,524
5,1705,1455,1350,704,345,706,919,806,1332,1221,771,782,452,455,4587,508
6,1771,1739,1938,845,715,883,1135,939,1362,1229,845,519,341,485,4126,301
7,1926,1475,1348,478,406,739,855,598,1083,912,694,316,346,138,3062,552
8,1791,1494,1445,543,442,654,1075,1016,1360,961,799,331,556,648,3184,488
total,15016,12801,11958,4947,3851,6330,8663,7591,10694,8579,6139,4047,3695,3424,28175,4477


In [6]:
# double check the groupings of smaller retinotopic ROIs into bigger ones

ret_group_names = roi_utils.ret_group_names
ret_group_inds = roi_utils.ret_group_inds

ret, face, place, body,  = roi_utils.load_roi_label_mapping(subject=1)

for gg, group_name in enumerate(ret_group_names):
    print(group_name)
    include = np.isin(ret[0], ret_group_inds[gg])
    print(np.array(ret[1])[include])

V1
['V1v' 'V1d']
V2
['V2v' 'V2d']
V3
['V3v' 'V3d']
hV4
['hV4']
VO1-2
['VO1' 'VO2']
PHC1-2
['PHC1' 'PHC2']
LO1-2
['LO2' 'LO1']
TO1-2
['TO2' 'TO1']
V3ab
['V3B' 'V3A']
IPS0-1
['IPS0' 'IPS1']
IPS2-5
['IPS2' 'IPS3' 'IPS4' 'IPS5']
SPL1
['SPL1']
FEF
['FEF']


In [20]:
roi_def = roi_utils.multi_subject_roi_def(np.arange(1,9), skip_areas = [4,5,7,11,12, 19,20, 22, 24])
n_total = roi_def.get_sizes()[np.newaxis,:]
roi_df = pd.DataFrame(data=n_total, columns=roi_def.roi_names, \
                      index=['total'])
roi_df

,V1,V2,V3,hV4,LO1-2,V3ab,IPS0-1,IPS2-5,OPA,PPA,RSC,OFA,FFA-1,FFA-2,EBA,FBA-2
total,15016,12801,11958,4947,3851,6330,8663,7591,10694,8579,6139,4047,3695,3424,28175,4477


In [21]:
roi_def = roi_utils.multi_subject_roi_def(np.arange(1,9), skip_areas = [])
n_total = roi_def.get_sizes()[np.newaxis,:]
roi_df = pd.DataFrame(data=n_total, columns=roi_def.roi_names, \
                      index=['total'])
roi_df.iloc[:,0:12]

,V1,V2,V3,hV4,VO1-2,PHC1-2,LO1-2,TO1-2,V3ab,IPS0-1,IPS2-5,SPL1
total,15016,12801,11958,4947,4100,3069,3851,3258,6330,8663,7591,1396


In [22]:
roi_df.iloc[:,12:24]

,FEF,OPA,PPA,RSC,OFA,FFA-1,FFA-2,mTL-faces,aTL-faces,EBA,FBA-1,FBA-2
total,665,10694,8579,6139,4047,3695,3424,619,1255,28175,3286,4477
